In [1]:
import json
import pandas as pd
import duckdb

In [2]:
with open('relation.json', 'r') as f:
    relation = json.loads(f.read())

In [3]:
with open('way.json', 'r') as f:
    way = json.loads(f.read())

In [4]:
[el.get('tags').get('name') for el in relation.get('elements') if el.get('tags', {}).get('name')]

['Domaine Vieux Moulin',
 "Rue de l'Abbaye",
 'Rue de la Limite',
 'Rue du 8 Mai',
 'Place Verte']

In [5]:
names = [
    {
        'type': el.get('type'),
        'id': el.get('id'),
        'name': el.get('tags').get('name'),
        'gender': '',
        'person': '',
        'description': '',
    }
    for el in way.get('elements') if el.get('tags', {}).get('name')
]

In [6]:
df_way = pd.DataFrame(names)

In [7]:
df_prenoms = pd.read_csv('Prenoms.csv', sep=';')

In [8]:
df_prenom_clean = duckdb.sql("""
    SELECT
        *,
        REPLACE(REPLACE("01_prenom", ' (1)', ''), ' (2)', '') AS prenom
    FROM
        df_prenoms
    WHERE
        "01_prenom" NOT LIKE 'rue'
""").df()

In [9]:
df_prenoms_unique = duckdb.sql("""
    SELECT DISTINCT ON (prenom)
        *
    FROM
        df_prenom_clean
    WHERE
        prenom IS NOT NULL
    ORDER BY
        prenom,
        "04_fréquence" DESC
    
""").df()

In [10]:
df_prenoms_unique

,01_prenom,02_genre,03_langage,04_fréquence,prenom
0,'abbas,m,arabic,0.0,'abbas
1,'abla,f,arabic,0.0,'abla
2,'aisha,f,arabic,0.0,'aisha
3,'aziz,m,arabic,0.0,'aziz
4,'isam,m,arabic,0.0,'isam
...,...,...,...,...,...
11503,þórbjörn,m,ancient scandinavian,0.0,þórbjörn
11504,þórdís,f,ancient scandinavian,0.0,þórdís
11505,þórir,m,icelandic,0.0,þórir
11506,þórr,m,norse mythology,0.0,þórr


In [11]:
df_genre = pd.DataFrame({'02_genre': ['m', 'f', 'f,m', 'm,f'], 'gender': ['M', 'F', 'F', 'M']})

In [12]:
df_final = duckdb.sql("""
    SELECT DISTINCT ON (w.id)
        w.type,
        w.id,
        w.name,
        g.gender,
        w.person,
        w.description,
        p.prenom,
        p."02_genre"
    FROM
        df_way AS w
    LEFT JOIN
        df_prenoms_unique AS p
        ON strip_accents(w.name) ILIKE '% ' || strip_accents(p.prenom) || ' %'
    LEFT JOIN
        df_genre AS g
        ON p."02_genre" = g."02_genre"
    ORDER BY
        w.id,
        p."04_fréquence" DESC
""").df()

In [13]:
df_final

,type,id,name,gender,person,description,prenom,02_genre
0,way,4309822,Rue du Tilloi,NaN,,,NaN,NaN
1,way,4309823,Rue du Muturnia,NaN,,,NaN,NaN
2,way,4309827,Grand Ring de Charleroi,NaN,,,NaN,NaN
3,way,4309829,Chaussée de Charleroi,NaN,,,NaN,NaN
4,way,4405030,Rue Dominique Seret,F,,,dominique,"f,m"
...,...,...,...,...,...,...,...,...
10500,way,1173652014,Rue d'Heppignies,NaN,,,NaN,NaN
10501,way,1173968897,Sentier de Buzet,NaN,,,NaN,NaN
10502,way,1174678403,Rue du Cazier,NaN,,,NaN,NaN
10503,way,1175521590,Rue de Beaumont,NaN,,,NaN,NaN


In [14]:
df_final.to_csv('data.csv')